# Flow Shop Scheduling Problem

--------------------
* Developed by _Keivan Tafakkori_
* Date: 18-4-2023
* Contact: https://www.linkedin.com/in/keivan-tafakkori/
* Github: https://github.com/ktafakkori
--------------------

### Packages

In [1]:
from feloopy import *

### Setting

In [2]:
problem_name = 'fssp'
solution_method = 'exact'
interface = 'pulp'
solver = 'highs'
key = 0

### Model

In [3]:
#Environment
m = target_model(solution_method, problem_name, interface, key=key)

#Dataset
w = [0.1, 0.4, 0.15, 0.35] #Priority weight of each job
p = [[  7,   3,    9,    4],
     [  7,   3,    9,    4]] #Processing time of each job on each machine
s = [5, 2] #Setup times of the machines


#Sets
I = range(len(p[0])) #Set of jobs
J = range(len(I)) #Set of positions
K = range(len(p)) #Set of machines

#Variables
x = m.bvar('x', [I,J])
c = m.pvar('c', [J,K])
d = m.pvar('d', [J,K])

#Objective
m.obj(sum(w[j]*c[j,1] for j in J))

#Constraints
for i in I: 
    m.con(sum(x[i,j] for j in J) == 1)

for j in J: 
    m.con(sum(x[i,j] for i in I) == 1)

for j in J: 
    if j!=0: 
        m.con(c[j,1] >= sum(x[i,j]*p[1][i] for i in I) + d[j,1])

m.con(c[0,1] == s[1] + sum(x[i,0]*p[1][i] for i in I) + c[0,0])

for j in J: 
    if j!=0: 
        m.con(c[j,0] >= c[j-1,0] + sum(x[i,j]*p[0][i] for i in I))

m.con(c[0,0] == s[0] + sum(x[i,0]*p[0][i] for i in I))

for j in J: 
    if j!=0: 
        m.con(d[j,1] >= c[j-1,1])

for j in J: 
    m.con(d[j,1] >= c[j,0])

#Solve
m.sol(['min'], solver)

Restricted license - for non-production use only - expires 2024-10-28


### Result

In [4]:
#Report
m.report()

#Display
for i,j in sets(I,J):
    if m.get(x[i,j])!=0:
        print(f"x{i}{j} = {m.get(x[i,j])}")

for j,k in sets(J,K):
    if m.get(c[j,k])!=0:
        print(f"c{j}{k} = {m.get(c[j,k])}")

for j,k in sets(J,K):
    if m.get(d[j,k]):
        print(f"d{j}{k} = {m.get(d[j,k])}")


~~~~~~~~~~~~~~
FELOOPY v0.2.4
~~~~~~~~~~~~~~

~~~~~~~~~~~
DATE & TIME
~~~~~~~~~~~
2023-04-18 23:27:59
Tue, Apr 18, 2023

~~~~~~~~~~~~
PROBLEM INFO
~~~~~~~~~~~~
| info      | detail   | variable   | count [cat,tot]   | other      | count [cat,tot]    |
|-----------|----------|------------|-------------------|------------|--------------------|
| model     | fssp     | positive   | [2, 16]           | objective  | [1, 1]             |
| interface | pulp     | binary     | [1, 16]           | constraint | [1, 23]            |
| solver    | highs    | integer    | [0, 0]            |            |                    |
| direction | ['min']  | free       | [0, 0]            |            |                    |
| method    | exact    | tot        | [3, 32]           |            |                    |
~~~~~~~~~~~~

~~~~~~~~~~
SOLVE INFO
~~~~~~~~~~
status:  Optimal
objective:  24.950000000000003
cpu time [pulp]:  71981.69999992388 (microseconds) 00:00:00 (h, m, s)
~~~~~~~~~~~

~~~~~~~~~~
MODEL 